<a href="https://colab.research.google.com/github/srmty09/Deep-Learning/blob/main/Fine_Tuning/unsloth_finetuning_blog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning Using Unsloth.

1. Model Loading.

In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.7/405.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/

In [1]:
from unsloth import FastLanguageModel
# fastlanguage model is where all the language model present.
import torch
max_seq_len = 512
load_in_4bit = True # for using 4bit quantized model.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Some of 4 bit quantized models:
1. "unsloth/Meta-Llama-3.1-8B-bnb-4bit"
2. "unsloth/Mistral-Small-Instruct-2409"
3. "unsloth/phi-4"
4. "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"

In [3]:
model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/phi-4", # which model we want to use
    max_seq_length = max_seq_len,
    load_in_4bit = load_in_4bit
    )

==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Model names ending with *unsloth-bnb-4bit* indicate they are unsloth dynamic 4bit models, these models are slightly more vram than standard BitsBytes 4bit models but offer better accuracy.

# Dataset
___

- we need to create a dataset usually with 2 columns Question and answer.
- we can create synthetically generated data and structure the dataset using chatgpt.
- we need to know the purpose of dataset, to know for what cause we need the dataset and how we will use it.

## Chat Template with Unsloth


In [4]:
from unsloth.chat_templates import CHAT_TEMPLATES
print(list(CHAT_TEMPLATES.keys()))

['unsloth', 'zephyr', 'chatml', 'mistral', 'llama', 'vicuna', 'vicuna_old', 'vicuna old', 'alpaca', 'gemma', 'gemma_chatml', 'gemma2', 'gemma2_chatml', 'llama-3', 'llama3', 'phi-3', 'phi-35', 'phi-3.5', 'llama-3.1', 'llama-31', 'llama-3.2', 'llama-3.3', 'llama-32', 'llama-33', 'qwen-2.5', 'qwen-25', 'qwen25', 'qwen2.5', 'phi-4', 'gemma-3', 'gemma3', 'qwen-3', 'qwen3', 'gemma-3n', 'gemma3n', 'gpt-oss', 'gptoss', 'qwen3-instruct', 'qwen3-thinking', 'lfm-2', 'starling', 'yi-chat']


In [49]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [50]:
def formatting_prompts_func(examples):
  convos = examples["messages"]
  texts = [tokenizer.apply_chat_template(convo,tokenize=False,add_generation_prompt=False) for convo in convos]
  return {"text":texts,}

In [51]:
examples = {
    "messages": [
        [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is Bayes theorem?"},
            {"role": "assistant", "content": "Bayes theorem describes conditional probability."}
        ],
        [
            {"role": "system", "content": "You are a math tutor."},
            {"role": "user", "content": "Explain eigenvalues."},
            {"role": "assistant", "content": "Eigenvalues measure scaling of linear transforms."}
        ]
    ]
}


In [52]:
formatting_prompts_func(examples)

{'text': ['<|endoftext|><start_of_turn>user\nYou are a helpful assistant.\n\nWhat is Bayes theorem?<end_of_turn>\n<start_of_turn>model\nBayes theorem describes conditional probability.<end_of_turn>\n',
  '<|endoftext|><start_of_turn>user\nYou are a math tutor.\n\nExplain eigenvalues.<end_of_turn>\n<start_of_turn>model\nEigenvalues measure scaling of linear transforms.<end_of_turn>\n']}

In [53]:
from datasets import load_dataset
dataset = load_dataset("HuggingFaceH4/ultrachat_200k",split = "train_sft")

# dataset.map is used to map the dataset to the required format to train the model.
dataset = dataset.map(formatting_prompts_func,batched = True)

Map:   0%|          | 0/207865 [00:00<?, ? examples/s]

In [54]:
print(dataset)

Dataset({
    features: ['prompt', 'prompt_id', 'messages', 'text'],
    num_rows: 207865
})


In [56]:
dataset[1]["text"]

'<|endoftext|><start_of_turn>user\nWhich famous landmarks should I visit in London, beyond the usual ones?<end_of_turn>\n<start_of_turn>model\n1. Leadenhall Market - a beautiful indoor market with stunning Victorian architecture, also used as a filming location in the Harry Potter films.\n\n2. St. Dunstan in the East - a ruined church in the middle of the city that has been turned into a beautiful public garden.\n\n3. The Monument - a 202-foot-tall column commemorating the Great Fire of London, with a staircase leading to a viewing platform offering great views of the city.\n\n4. The Camden Town Markets - an eclectic collection of markets offering food, fashion, and vintage items, plus live music and street performers.\n\n5. Novelist\'s House - the former home of Charles Dickens, now a museum dedicated to his life and works.\n\n6. The Old Operating Theatre - a museum housed in the oldest surviving operating theatre in Europe, with exhibits on the history of surgery and medical practice

# some datasets
1. Alpaca Dataset
2. HuggingFaceH4/ultrachat_200k
3.

In [58]:
dataset = load_dataset("vicgalle/alpaca-gpt4",split = "train")
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})


In [60]:
dataset[0]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for 

In [ ]:
dataset = dataset.map(formatting_prompts_func,batched = True)

# Finetuning Llama-3!!


In [3]:
!pip install "unsloth[colab-new] @ git+http://github.com/unslothai/unsloth.git"
!pip uninstall -y xformers
!pip install trl==0.8.6 --no-deps

  Cloning http://github.com/unslothai/unsloth.git to /tmp/pip-install-_rsi7riq/unsloth_6957be2038b54c2b85c2543da7f9ab78
  Running command git clone --filter=blob:none --quiet http://github.com/unslothai/unsloth.git /tmp/pip-install-_rsi7riq/unsloth_6957be2038b54c2b85c2543da7f9ab78
  Resolved http://github.com/unslothai/unsloth.git to commit 485be2a33e52b83bb4bbf45743f74e853ef038da
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.8.6-py3-none-any.whl (245 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.24.0
    Uninstalling trl-0.24.0:
      Successfully uninstalled trl-0.24.0


In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [41]:
# peft: parameter efficient fine tuning

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj",
                      "gate_proj","up_proj","down_proj"],
    lora_alpha = 16,
    lora_dropout=0,
    bias = "none",
    use_gradient_checkpointing= "unsloth",
    random_state = 3407,
    use_rslora=False,
    loftq_config=None
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [42]:
from datasets import load_dataset
dataset = load_dataset("vicgalle/alpaca-gpt4",split = "train")
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})


In [43]:
# from unsloth import to_sharegpt
# dataset = to_sharegpt(
#     dataset,
#     merged_prompt = "{instruction}[[\nYour input is:\n{input}]]",
#     output_column_name="output",
#     conversation_extension=3,
# )

In [44]:
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})


In [45]:
dataset[0]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for 

In [46]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

In [50]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset.select(range(10000)),
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        remove_unused_columns = False,
    ),
)


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 882 | Num Epochs = 1 | Total steps = 111
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,1.154200
2,1.191800
3,1.223100
4,1.175600
5,1.174900
6,1.246000
7,1.135600
8,1.193700
9,1.234000
10,1.135500
